## Setup

Abaixo é necessário instalar algumas bibliotecas, incluindo LlamaIndex🦙, para conseguir executar este laboratório.

In [ ]:
%pip install llama-index-llms-oci-genai

In [ ]:
!pip install llama-index

You will also need to install the OCI sdk

In [ ]:
!pip install -U oci

In [ ]:
!pip install openpyxl

## Iniciando as bibliotecas e classes

Aqui é apenas necessário executar sem efetuar nenhuma alteração. As bibliotecas e classes serão carregadas para possibilitar a conexão com o OCI Generative AI para utilizar o LLM Cohere Command-R+.

Os comandos abaixo das importações são as configurações necessárias para utilizar o LLM Cohere Command-R+: 

In [6]:
import os
from llama_index.llms.oci_genai import OCIGenAI
from llama_index.core.llms import ChatMessage
import pandas as pd


compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID'] 

command_r_plus = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceya7ozidbukxwtun4ocm4ngco2jukoaht5mygpgr6gq2lgq"

llm = OCIGenAI(
    model=command_r_plus,
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    provider="cohere",  # e.g., "cohere" or "meta"
    compartment_id=compartment_id,
    context_size="16000"
)

### Importando o Arquivo Com Tweets

In [7]:
premier_tweets = pd.read_excel("~/OCI_Winning_with_AI/Análise de Sentimentos/tweets_premier_league_footballersv3_br.xlsx", sheet_name='tweet_premier_league')

In [ ]:
premier_tweets

In [9]:
tweets=list(premier_tweets["texto"])

In [ ]:
tweets

## Análise de Sentimento com OCI Generative AI

Abaixo segue um exemplo de tweet para teste e entendimento de como é feito o prompt, a chamada do LLM e a resposta do mesmo:

In [47]:
frase = '''
Não é de admirar que eles sejam amaldiçoados
'''

In [48]:
preamble = 'Você é um agente especialista em analizar o sentimento das frases escritas abaixo. Sua tarefa é responder sobre estas frases escritas por jogadores de futebol apenas e somente uma entre as seguintes opções: Positivo, Negativo ou Neutro'
    
prompt = f'''
Frase: {frase};

Por favor responda com muita atenção, pois isto é muito importante para minha carreira!
'''

messages = [
        ChatMessage(
            role="system", content=preamble),
        ChatMessage(
            role="user", content=prompt),
        ]

In [ ]:
resp = llm.chat(messages)

print(resp)

In [ ]:
resp.message.content

## Laço para Análise de Todos os tweets

Abaixo está o código do laço que fará a análise de sentimentos de todos os tweets:

In [ ]:
genAI_emotion=[]

for frase in tweets:
    tweet=frase
    preamble='''Você é um agente especialista em analizar o sentimento das frases escritas abaixo. Sua tarefa é responder sobre estas frases escritas por jogadores de futebol apenas e somente uma entre as seguintes opções: "Positivo", "Negativo" ou "Neutro", por favor não inserir nenhum caracter adicional como "." por exemplo'''
    prompt = f'''
    Tweet: {tweet};

    Por favor responda com muita atenção, pois isto é muito importante para minha carreira!
    '''
    messages = [
        ChatMessage(
            role="system", content=preamble),
        ChatMessage(
            role="user", content=prompt),
        ]
    resp = llm.chat(messages)
    genAI_emotion.append([frase, resp.message.content])

genAI_emotion

## Organizando as Análises

Nas células abaixo são ajustadas os dados em dataframe e acrescentado no dataframe importado inicialmente do arquivo xlsx: 

In [38]:
df_genAI_emotion=pd.DataFrame(genAI_emotion)

In [ ]:
df_genAI_emotion.rename(columns={0: "tweet", 1: "genAI_emotion"}, inplace=True)
df_genAI_emotion

In [ ]:
df_genAI_emotion["genAI_emotion"].value_counts()

In [42]:
premier_tweets = pd.concat([premier_tweets.reset_index(drop=True),df_genAI_emotion["genAI_emotion"]],axis=1)

In [ ]:
premier_tweets

## Dados Finais

Por último, os dados são gravados num arquivo csv:

In [44]:
premier_tweets.to_csv('~/Winning_AI/tweets_premier_league_sentiment_analyses.csv',index=False)